In [63]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx + 1
    series = series.map(my_dict)

    return series

In [3]:
import pandas as pd
import numpy as np
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_all = pd.concat([df_train, df_test], axis = 0)
df_all.shape, df_train.shape, df_test.shape

((30000, 23), (20000, 23), (10000, 22))

In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      30000 non-null  object 
 1   Age                     30000 non-null  int64  
 2   Gender                  30000 non-null  object 
 3   Education_Status        30000 non-null  object 
 4   Employment_Status       30000 non-null  object 
 5   Working_Week (Yearly)   30000 non-null  int64  
 6   Industry_Status         30000 non-null  object 
 7   Occupation_Status       30000 non-null  object 
 8   Race                    30000 non-null  object 
 9   Hispanic_Origin         30000 non-null  object 
 10  Martial_Status          30000 non-null  object 
 11  Household_Status        29999 non-null  object 
 12  Household_Summary       30000 non-null  object 
 13  Citizenship             30000 non-null  object 
 14  Birth_Country           30000 non-null  obje

In [5]:
df_all.describe(include = 'int64')

,Age,Working_Week (Yearly),Gains,Losses,Dividends
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,35.736667,35.022633,394.979467,39.278000,129.001200
std,17.979206,22.232172,4264.589678,275.037897,1327.460917
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,7.000000,0.000000,0.000000,0.000000
50%,35.000000,52.000000,0.000000,0.000000,0.000000
75%,47.000000,52.000000,0.000000,0.000000,0.000000
max,90.000000,52.000000,99999.000000,4356.000000,99999.000000


In [6]:
# 이상치 99999 제거한 후 그 다음 최댓값으로 대체
df_all['Gains'].replace(99999, np.nan, inplace = True)
df_all['Dividends'].replace(99999, np.nan, inplace = True)

df_all['Gains'].fillna(df_all['Gains'].max(), inplace = True)
df_all['Dividends'].fillna(df_all['Dividends'].max(), inplace = True)

In [7]:
df_all.describe(include = ['int64', 'float64'])

,Age,Working_Week (Yearly),Gains,Losses,Dividends,Income
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,20000.000000
mean,35.736667,35.022633,289.533067,39.278000,127.501233,554.565250
std,17.979206,22.232172,2029.459535,275.037897,1236.946305,701.553155
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,7.000000,0.000000,0.000000,0.000000,0.000000
50%,35.000000,52.000000,0.000000,0.000000,0.000000,500.000000
75%,47.000000,52.000000,0.000000,0.000000,0.000000,875.000000
max,90.000000,52.000000,34095.000000,4356.000000,55000.000000,9999.000000


In [8]:
# 레이블 인코딩할 칼럼들
label_columns = df_all.select_dtypes(include = 'object').columns.drop('ID')

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

df_all.drop(columns='ID', inplace = True)
df_all.columns = df_all.columns.str.replace(" ", '_')

df_train = df_all.iloc[:len(df_train)]
df_test = df_all.iloc[len(df_train):]
df_test.drop(columns = 'Income', axis = 1, inplace = True)

C:\Users\win\AppData\Local\Temp\ipykernel_1584\2584239544.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(columns = 'Income', axis = 1, inplace = True)


In [9]:
df_test

,Age,Gender,Education_Status,Employment_Status,Working_Week_(Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,...,Household_Summary,Citizenship,Birth_Country,Birth_Country_(Father),Birth_Country_(Mother),Tax_Status,Gains,Losses,Dividends,Income_Status
0,79,2,10,1,0,15,15,5,1,6,...,5,3,40,41,41,6,0.0,0,0.0,2
1,47,2,8,1,0,15,15,5,9,6,...,1,3,40,40,40,5,0.0,0,0.0,2
2,18,1,10,1,52,20,12,5,1,6,...,1,3,40,40,40,6,0.0,0,0.0,2
3,39,1,2,3,30,13,12,5,1,2,...,8,3,40,40,40,3,0.0,0,0.0,3
4,6,2,4,1,0,15,15,5,7,6,...,3,3,40,40,40,5,0.0,0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,31,2,3,1,52,16,13,5,1,6,...,5,3,40,40,40,6,0.0,0,0.0,2
9996,27,2,5,1,52,20,5,5,1,2,...,5,3,40,40,40,3,0.0,0,0.0,2
9997,18,2,13,1,7,20,12,2,1,6,...,1,3,40,40,40,5,0.0,0,0.0,2
9998,9,2,4,1,0,15,15,5,1,6,...,3,3,40,40,40,5,0.0,0,0.0,2


In [61]:
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(df_train.drop(columns = 'Income', axis = 1), df_train['Income'], test_size= 0.3, random_state = 42)

In [21]:
lgbm = LGBMRegressor(random_state = 42, verbose = 0)

param_grid = {'n_estimators' : np.arange(200, 601, 100), 'learning_rate' : np.arange(0.03, 0.12, 0.02),
              'subsample' : [0.8, 0.9, 1], 'colsample_bytree' : [0.8, 0.9, 1]}

grid_lgbm = GridSearchCV(lgbm, param_grid, verbose = True, scoring='neg_root_mean_squared_error')
grid_lgbm.fit(X_train, y_train)

Fitting 5 folds for each of 225 candidates, totalling 1125 fits


GridSearchCV(estimator=LGBMRegressor(random_state=42, verbose=0),
             param_grid={'colsample_bytree': [0.8, 0.9, 1],
                         'learning_rate': array([0.03, 0.05, 0.07, 0.09, 0.11]),
                         'n_estimators': array([200, 300, 400, 500, 600]),
                         'subsample': [0.8, 0.9, 1]},
             scoring='neg_root_mean_squared_error', verbose=True)

In [22]:
grid_lgbm.best_params_

{'colsample_bytree': 0.8,
 'learning_rate': 0.03,
 'n_estimators': 200,
 'subsample': 0.8}

In [62]:
lgbm = LGBMRegressor(random_state = 42, verbose = 0, colsample_bytree = 0.8, 
                     learning_rate = 0.03, n_estimators = 200, subsample = 0.8)

lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)
# y_pred[y_pred < 0] = 0
print('rmse :', np.sqrt(mean_squared_error(y_pred, y_test)))

rmse : 579.4922207005494


In [24]:
y_pred = grid_lgbm.predict(X_test)
y_pred = np.expm1(y_pred)
print('rmse :', np.sqrt(mean_squared_error(y_pred, y_test)))

rmse : 759.4496676254934


In [15]:
# y_pred = grid_lgbm.predict(df_test)
y_pred = lgbm.predict(df_test)
y_pred

array([ -9.29208148, -17.52285151, 442.90653313, ..., 360.10504305,
         5.36444684, 649.62809807])

In [16]:
submission = pd.read_csv('sample_submission.csv')
submission['Income'] = y_pred
submission.to_csv('./baseline_submission.csv', index=False)

In [17]:
submission

,ID,Income
0,TEST_0000,-9.292081
1,TEST_0001,-17.522852
2,TEST_0002,442.906533
3,TEST_0003,594.067553
4,TEST_0004,2.721028
...,...,...
9995,TEST_9995,880.216055
9996,TEST_9996,783.084140
9997,TEST_9997,360.105043
9998,TEST_9998,5.364447


## pycaret

In [24]:
pip install pycaret

                                              0.0/485.9 kB ? eta -:--:--
     ------------------------------------- 485.9/485.9 kB 15.3 MB/s eta 0:00:00
                                              0.0/10.6 MB ? eta -:--:--
     -----                                    1.6/10.6 MB 48.3 MB/s eta 0:00:01
     -----------                              3.1/10.6 MB 48.6 MB/s eta 0:00:01
     -----------------                        4.6/10.6 MB 41.4 MB/s eta 0:00:01
     ----------------------                   6.1/10.6 MB 42.8 MB/s eta 0:00:01
     ----------------------------             7.5/10.6 MB 43.8 MB/s eta 0:00:01
     ---------------------------------        9.0/10.6 MB 41.1 MB/s eta 0:00:01
     --------------------------------------  10.5/10.6 MB 40.9 MB/s eta 0:00:01
     --------------------------------------- 10.6/10.6 MB 36.2 MB/s eta 0:00:00
                                              0.0/160.5 kB ? eta -:--:--
     ---------------------------------------- 160.5/160.5 kB ?

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.


In [ ]:
from pycaret.regression import *

reg_setup = setup(data=df_train, target='Income')
compare_models(sort = 'MSE')

In [ ]:
# extra tree 를 기본모델로 사용 - 추후 앙상블에 여러 모델 사용하자

In [28]:
lgbm_model = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,315.4206,287124.8909,535.8404,0.3003,3.0968,0.3154
1,346.4105,447949.4454,669.2903,0.2328,3.0558,0.3268
2,324.4778,261500.9345,511.3716,0.2995,3.0881,0.3624
3,340.7171,340960.9598,583.9186,0.3056,3.0509,0.3084
4,327.6572,292624.6058,540.9479,0.3281,3.0711,0.3147
5,328.3274,296769.9516,544.7660,0.3170,3.1098,0.3132
6,349.7388,453406.4727,673.3546,0.2347,3.0863,0.2956
7,325.6925,302392.1898,549.9020,0.3031,3.2025,0.2969
8,337.9864,434888.7023,659.4609,0.2461,3.0778,0.2850


In [30]:
# 모델 튜닝
tuned_lgbm = tune_model(lgbm_model)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [32]:
tuned_lgbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.15,
 'max_depth': -1,
 'min_child_samples': 91,
 'min_child_weight': 0.001,
 'min_split_gain': 0.9,
 'n_estimators': 300,
 'n_jobs': -1,
 'num_leaves': 4,
 'objective': None,
 'random_state': 2045,
 'reg_alpha': 0.2,
 'reg_lambda': 0.01,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'feature_fraction': 0.8,
 'bagging_freq': 3,
 'bagging_fraction': 1.0}

In [ ]:
tuned_lgbm